In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from proj1_helpers import *
from Hugues import *


Data exploration
===

Load data

In [2]:
from proj1_helpers import *

labels, raw_data, indices = load_csv_data('train.csv', sub_sample=False)

labels_te, raw_data_te, indices_te = load_csv_data('test.csv', sub_sample=False)

Divide data

In [3]:
labels0, data0, labels1, data1, labels2, data2 = divide_data(labels, raw_data)
labels0_te, data0_te, labels1_te, data1_te, labels2_te, data2_te = divide_data(labels_te, raw_data_te)

print('Train:', '0 jet', data0.shape, ', 1 jet', data1.shape, ', 2 jet', data2.shape)
print('Test:', '0 jet', data0_te.shape, ', 1 jet', data1_te.shape, ', 2 jet', data2_te.shape)

Train: 0 jet (99913, 30) , 1 jet (77544, 30) , 2 jet (72543, 30)
Test: 0 jet (227458, 30) , 1 jet (175338, 30) , 2 jet (165442, 30)


Standardize data

In [4]:
std_data0, mean0, std0 = standardize_train(data0)
std_data0_te = standardize_test(data0_te, mean0, std0)

std_data1, mean1, std1 = standardize_train(data1)
std_data1_te = standardize_test(data1_te, mean1, std1)

std_data2, mean2, std2 = standardize_train(data2)
std_data1_te = standardize_test(data1_te, mean2, std2)

Estiamtion of feature 0

In [5]:
data, weights_train = column_estimation_train(std_data2)

IndexError: arrays used as indices must be of integer (or boolean) type

Estimate whole features
===

In [11]:
# Estimation of columns for train and test set
estimated_data,nan_columns_train,weights_train = column_estimation_train(nan_data)
estimated_data_te = column_estimation_test(nan_data_te,nan_columns_train,weights_train)


Columns containing NaN [ 0  4  5  6 12 23 24 25 26 27 28]


In [12]:
# Standardization of train and test set
std_data, mean_train, std_train = standardize_train(estimated_data)
std_data_te = standardize_test(estimated_data_te, mean_train, std_train)

Find best parameters
===

In [13]:
#Parameters
degrees = [10,11]
lambdas = np.logspace(-8, -10, 3)
k_fold = 10
seed = 42

k_idx = build_k_indices(labels, k_fold, seed)
loss_te = np.ones((len(degrees),len(lambdas)))
scores = np.ones((len(degrees),len(lambdas)))

for degree_idx, degree in enumerate(degrees):
    for lambda_idx, lambda_ in enumerate(lambdas):
        _ ,loss_te[degree_idx, lambda_idx], scores[degree_idx, lambda_idx]= cross_validation_(labels, std_data, k_idx, k_fold, lambda_, degree)
        print('Degree:', degrees[degree_idx], 'Lambda:', lambdas[lambda_idx])
        print('Score:', scores[degree_idx, lambda_idx])
        print('Loss:', loss_te[degree_idx, lambda_idx])

Degree: 10 Lambda: 1e-08
Score: 0.8122
Loss: 0.09285
Degree: 10 Lambda: 1e-09
Score: 0.81216
Loss: 0.092854
Degree: 10 Lambda: 1e-10
Score: 0.81216
Loss: 0.09285600000000001
Degree: 11 Lambda: 1e-08
Score: 0.81392
Loss: 0.09202800000000001
Degree: 11 Lambda: 1e-09
Score: 0.81396
Loss: 0.092016
Degree: 11 Lambda: 1e-10
Score: 0.81392
Loss: 0.09203800000000001


In [14]:
ratio = scores/loss_te
best_HP_idx = np.unravel_index(np.argmax(ratio), np.shape(ratio))
best_degree = degrees[best_HP_idx[0]]
best_lambda = lambdas[best_HP_idx[1]]

best_score = scores[best_HP_idx[0], best_HP_idx[1]]
best_loss = loss_te[best_HP_idx[0], best_HP_idx[1]]

print('Best degree:', best_degree, 'Best lambda:', best_lambda, 'Best score:', best_score, 'Best loss', best_loss)

Best degree: 11 Best lambda: 1e-09 Best score: 0.81396 Best loss 0.092016


Prediction
===

In [15]:
# Train model to get weights
poly_std_data = build_poly(std_data, best_degree)
weights, loss = ridge_regression(labels, poly_std_data, best_lambda)

#Predict on test
poly_std_data_te = build_poly(std_data_te, best_degree)
y_pred_te = predict_labels(weights, poly_std_data_te)

create_csv_submission(indices_te, y_pred_te, 'ridge_final.csv')